# Creating,Learning and following these steps:
    1.Importing necessary libraries
    2.Creating S3 bucket
    3.Mapping train and test data in S3
    4.Mapping the output path of models in S3

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [2]:
bucket_name= 'gurmanfirstapplicationbucket'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [3]:
s3 = boto3.resource('s3')
try:
    if my_region =='us-east-1':
        s3.create_bucket(Bucket=bucket_name)
        print("S3 created bucket successfully")
except Exception as e:
    print("S3 Error",e )

S3 created bucket successfully


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [4]:
#set an output path where the trained model will be saved
prefix='xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://gurmanfirstapplicationbucket/xgboost-as-a-built-in-algo/output


# Downloading and storing the dataset in S3

In [5]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv",
                               "bank_clean.csv")
    print("Success: Downloaded the dataset into the instance")
except Exception as e:
    print("Data load error",e)

try:
    model_data = pd.read_csv("./bank_clean.csv",index_col=0)
    print("Success: Data loaded into the dataframe")
except Exception as e:
    print("Data load error",e)

Success: Downloaded the dataset into the instance
Success: Data loaded into the dataframe


In [6]:
#Train-test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state = 1720), [int(0.7 * len(model_data))])
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


In [7]:
train_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
511,43,2,999,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,1,0
30587,38,5,999,0,1,0,0,1,0,0,...,0,1,0,0,0,0,1,0,1,0
25061,33,1,999,1,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0
7859,49,5,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
10180,39,6,999,0,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [8]:
#Saving train and test data into the buckets
#We start with the train data first
import os 
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [9]:
#Now puttin test data into buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

# Building Models Xgboot- Inbuilt Algorithm

In [14]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container =get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [15]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [16]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2022-04-29 14:17:16 Starting - Starting the training job...
2022-04-29 14:17:21 Starting - Launching requested ML instancesProfilerReport-1651241836: InProgress
.........
2022-04-29 14:19:17 Starting - Preparing the instances for training...
2022-04-29 14:19:45 Downloading - Downloading input data
2022-04-29 14:19:45 Training - Downloading the training image...
2022-04-29 14:20:17 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[14:20:17] 28831x59 matrix wi

# Deploy Machine Learning Models

In [18]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

---------!

# Prediction of the Test Data

In [23]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [24]:
predictions_array

array([0.08086   , 0.0429633 , 0.24573205, ..., 0.11874115, 0.06065132,
       0.05397812])

In [25]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.9%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10826)    34% (152)
Purchase        9% (1090)     66% (289) 



# Deleting the endpoints

In [26]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


[{'ResponseMetadata': {'RequestId': 'NW8ENXSRNKVWDGTC',
   'HostId': 'Zh6cS3AMXIDagssiKHRkjTtUb2omf1FpeQVFe0M5aqSCbQSbmbFojZuMjS5CaYAabHIudwtbcbg=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Zh6cS3AMXIDagssiKHRkjTtUb2omf1FpeQVFe0M5aqSCbQSbmbFojZuMjS5CaYAabHIudwtbcbg=',
    'x-amz-request-id': 'NW8ENXSRNKVWDGTC',
    'date': 'Fri, 29 Apr 2022 15:04:11 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-04-29-14-17-16-555/rule-output/ProfilerReport-1651241836/profiler-output/profiler-reports/BatchSize.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-04-29-14-17-16-555/rule-output/ProfilerReport-1651241836/profiler-output/profiler-report.html'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-04-29-14-17-16-555/profiler-output/system/incre